# ANALYSIS 

In [9]:
import cv2
import os
import mediapipe as mp
import numpy as np
import pandas as pd
from scipy.spatial import distance as dist

# --- 1. CONFIGURATION ---
input_base_path = r'C:\Users\arman\Desktop\FYP_ARMAN\Driver-Fatigue-Detection-Using-Vision-Based-Machine-Learning\CLEANED_DRIVER_FATIGUE'
output_base_path = r'C:\Users\arman\Desktop\FYP_ARMAN\Driver-Fatigue-Detection-Using-Vision-Based-Machine-Learning\EXTRACTED_FEATURES'
IMG_SIZE = (224, 224)
FRAME_SKIP = 5  # Process every 5th frame to avoid duplicate data

# --- 2. THRESHOLDS ---
MAR_YAWN_THRESHOLD = 0.5
MAR_TALK_THRESHOLD = 0.25
EAR_SLEEP_THRESHOLD = 0.22
EAR_NORMAL_THRESHOLD = 0.28

# --- 3. MEDIAPIPE & UTILITIES ---
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, refine_landmarks=True)

def calculate_ear(landmarks, eye_indices):
    v1 = dist.euclidean(landmarks[eye_indices[1]], landmarks[eye_indices[5]])
    v2 = dist.euclidean(landmarks[eye_indices[2]], landmarks[eye_indices[4]])
    h = dist.euclidean(landmarks[eye_indices[0]], landmarks[eye_indices[3]])
    return (v1 + v2) / (2.0 * h)

def calculate_mar(landmarks):
    v = dist.euclidean(landmarks[13], landmarks[14])
    h = dist.euclidean(landmarks[78], landmarks[308])
    return v / h

def get_head_pose(landmarks, img_w, img_h):
    # 3D model points
    model_points = np.array([
        (0.0, 0.0, 0.0),             # Nose tip
        (0.0, -330.0, -65.0),        # Chin
        (-225.0, 170.0, -135.0),     # Left eye left corner
        (225.0, 170.0, -135.0),      # Right eye right corner
        (-150.0, -150.0, -125.0),    # Left Mouth corner
        (150.0, -150.0, -125.0)      # Right mouth corner
    ])
    # 2D points from landmarks
    image_points = np.array([
        (landmarks[1][0] * img_w, landmarks[1][1] * img_h),
        (landmarks[152][0] * img_w, landmarks[152][1] * img_h),
        (landmarks[33][0] * img_w, landmarks[33][1] * img_h),
        (landmarks[263][0] * img_w, landmarks[263][1] * img_h),
        (landmarks[61][0] * img_w, landmarks[61][1] * img_h),
        (landmarks[291][0] * img_w, landmarks[291][1] * img_h)
    ], dtype="double")

    focal_length = img_w
    center = (img_w / 2, img_h / 2)
    camera_matrix = np.array([[focal_length, 0, center[0]], [0, focal_length, center[1]], [0, 0, 1]], dtype="double")
    dist_coeffs = np.zeros((4, 1))
    
    success, rot_vec, trans_vec = cv2.solvePnP(model_points, image_points, camera_matrix, dist_coeffs)
    rmat, _ = cv2.Rodrigues(rot_vec)
    angles, _, _, _, _, _ = cv2.RQDecomp3x3(rmat)
    return angles[0], angles[1], angles[2] # Pitch, Yaw, Roll

# --- 4. PROCESSING ENGINE ---
dataset_log = []

def process_video(video_path, label):
    cap = cv2.VideoCapture(video_path)
    video_name = os.path.basename(video_path).split('.')[0]
    dest_folder = os.path.join(output_base_path, label)
    os.makedirs(dest_folder, exist_ok=True)
    
    frame_idx = 0
    saved_count = 0

    while cap.isOpened():
        success, frame = cap.read()
        if not success: break
        
        if frame_idx % FRAME_SKIP == 0:
            h, w, _ = frame.shape
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = face_mesh.process(rgb)

            if results.multi_face_landmarks:
                landmarks = [(lm.x, lm.y) for lm in results.multi_face_landmarks[0].landmark]
                
                # Calculations
                ear_l = calculate_ear(landmarks, [33, 160, 158, 133, 153, 144])
                ear_r = calculate_ear(landmarks, [362, 385, 387, 263, 373, 380])
                avg_ear = (ear_l + ear_r) / 2.0
                mar = calculate_mar(landmarks)
                pitch, yaw, roll = get_head_pose(landmarks, w, h)

                # Logic for Saving
                save_flag = False
                if label == 'Yawning' and mar > MAR_YAWN_THRESHOLD: save_flag = True
                elif label == 'Microsleep' and avg_ear < EAR_SLEEP_THRESHOLD: save_flag = True
                elif label == 'Talking' and MAR_TALK_THRESHOLD < mar < MAR_YAWN_THRESHOLD: save_flag = True
                elif label == 'Normal' and avg_ear > EAR_NORMAL_THRESHOLD and mar < 0.2: save_flag = True

                if save_flag:
                    img_name = f"{video_name}_f{frame_idx}.jpg"
                    cv2.imwrite(os.path.join(dest_folder, img_name), cv2.resize(frame, IMG_SIZE))
                    saved_count += 1
                    
                    # Log data for later analysis
                    dataset_log.append({
                        'Filename': img_name,
                        'Label': label,
                        'EAR': avg_ear,
                        'MAR': mar,
                        'Pitch': pitch,
                        'Yaw': yaw,
                        'Roll': roll
                    })
        
        frame_idx += 1
    cap.release()
    return saved_count

# --- 5. EXECUTION LOOP ---
valid_labels = ['Yawning', 'Microsleep', 'Talking', 'Normal']

print("Starting Feature Extraction and Analysis...")
for root, dirs, files in os.walk(input_base_path):
    for file in files:
        if file.lower().endswith(('.mp4', '.avi')):
            # Identify label from folder path
            path_parts = os.path.normpath(root).split(os.sep)
            current_label = next((l for l in valid_labels if l.lower() in [p.lower() for p in path_parts]), None)
            
            if current_label:
                print(f"Processing {file} as {current_label}...")
                n = process_video(os.path.join(root, file), current_label)
                print(f"--> Saved {n} images.")

# Save CSV for Analysis
df = pd.DataFrame(dataset_log)
df.to_csv(os.path.join(output_base_path, 'feature_extraction_log.csv'), index=False)
print(f"\nSUCCESS: Data logged to {os.path.join(output_base_path, 'feature_extraction_log.csv')}")

Starting Feature Extraction and Analysis...
Processing P1042751_720.mp4 as Microsleep...
--> Saved 32 images.
Processing P1042756_720.mp4 as Microsleep...
--> Saved 83 images.
Processing P1042757_720.mp4 as Microsleep...
--> Saved 14 images.
Processing P1042762_720.mp4 as Microsleep...
--> Saved 143 images.
Processing P1042767_720.mp4 as Microsleep...
--> Saved 74 images.
Processing P1042772_720.mp4 as Microsleep...
--> Saved 19 images.
Processing P1042786_720.mp4 as Microsleep...
--> Saved 57 images.
Processing P1042787_720.mp4 as Microsleep...
--> Saved 12 images.
Processing P1042793_720.mp4 as Microsleep...
--> Saved 87 images.
Processing P1042797_720.mp4 as Microsleep...
--> Saved 173 images.
Processing P1043067_720.mp4 as Microsleep...
--> Saved 107 images.
Processing P1043068_720.mp4 as Microsleep...
--> Saved 58 images.
Processing P1043075_720.mp4 as Microsleep...
--> Saved 83 images.
Processing P1043089_720.mp4 as Microsleep...
--> Saved 238 images.
Processing P1043106_720.mp4 

# CLEANING YAWDD DATASET

In [5]:
import os
import shutil

# Your specific paths
yawdd_raw_path = r'C:\Users\arman\Desktop\FYP_ARMAN\Driver-Fatigue-Detection-Using-Vision-Based-Machine-Learning\YAWDD_DATASET'
nitymed_raw_path = r'C:\Users\arman\Desktop\FYP_ARMAN\Driver-Fatigue-Detection-Using-Vision-Based-Machine-Learning\NITYMED_DATASET'
combined_processed_path = r'C:\Users\arman\Desktop\FYP_ARMAN\Driver-Fatigue-Detection-Using-Vision-Based-Machine-Learning\CLEANED_DRIVER_FATIGUE'

# Supported video extensions
video_extensions = ('.mp4', '.avi', '.mkv', '.mov')

def organize_dataset(source_path, dataset_name):
    print(f"--- Processing {dataset_name} ---")
    count = 0
    skipped = 0
    
    # Check if source exists before starting
    if not os.path.exists(source_path):
        print(f"Error: Source path does not exist: {source_path}")
        return

    for root, dirs, files in os.walk(source_path):
        # Safety: Don't process the destination folder if it's inside the source
        if combined_processed_path in root:
            continue
            
        for file in files:
            if file.lower().endswith(video_extensions):
                file_lower = file.lower()
                root_lower = root.lower()
                
                # --- 1. EXCLUDE YAWDD DASH ---
                if dataset_name == "YawDD" and ('dash' in root_lower or 'dash' in file_lower):
                    skipped += 1
                    continue

                # --- 2. LABELING LOGIC ---
                if 'microsleep' in root_lower or 'microsleep' in file_lower:
                    label = 'Microsleep'
                elif 'yawn' in root_lower or 'yawn' in file_lower:
                    label = 'Yawning'
                elif 'talking' in file_lower or 'singing' in file_lower:
                    label = 'Talking'
                else:
                    label = 'Normal'
                
                # --- 3. ANGLE LOGIC ---
                if dataset_name == "NITYMED":
                    angle = 'aircond'
                else: 
                    # Only 'mirror' or 'unknown' will reach here due to dash exclusion
                    angle = 'mirror' if 'mirror' in root_lower or 'mirror' in file_lower else 'unknown'

                # --- 4. EXECUTE MOVE ---
                target_dir = os.path.join(combined_processed_path, label, angle)
                os.makedirs(target_dir, exist_ok=True)
                
                source_file = os.path.join(root, file)
                destination_file = os.path.join(target_dir, file)
                
                try:
                    shutil.move(source_file, destination_file)
                    count += 1
                except Exception as e:
                    print(f"Error moving {file}: {e}")

    print(f"Finished {dataset_name}. Moved: {count} | Skipped (Dash): {skipped}")

# Run the process
organize_dataset(yawdd_raw_path, "YawDD")
organize_dataset(nitymed_raw_path, "NITYMED")

print(f"\nSuccess! All files reorganized in: {combined_processed_path}")

--- Processing YawDD ---
Finished YawDD. Moved: 319 | Skipped (Dash): 29
--- Processing NITYMED ---
Finished NITYMED. Moved: 126 | Skipped (Dash): 0

Success! All files reorganized in: C:\Users\arman\Desktop\FYP_ARMAN\Driver-Fatigue-Detection-Using-Vision-Based-Machine-Learning\CLEANED_DRIVER_FATIGUE


# Frame Extraction & Feature Analysis

In [8]:
import cv2
import os
import mediapipe as mp
import numpy as np
from scipy.spatial import distance as dist

# --- Configuration ---
input_base_path = r'C:\Users\arman\Desktop\FYP_ARMAN\Driver-Fatigue-Detection-Using-Vision-Based-Machine-Learning\CLEANED_DRIVER_FATIGUE'
output_base_path = r'C:\Users\arman\Desktop\FYP_ARMAN\Driver-Fatigue-Detection-Using-Vision-Based-Machine-Learning\EXTRACTED_FEATURES'
IMG_SIZE = (224, 224)

# --- Reduction Settings ---
FRAME_SKIP = 5  # Only process every 5th frame to reduce data redundancy
MAX_FRAMES_PER_VIDEO = 100 # Optional cap to prevent one long video from dominating

# --- Thresholds ---
MAR_YAWN_THRESHOLD = 0.5
MAR_TALK_THRESHOLD = 0.25
EAR_SLEEP_THRESHOLD = 0.22
EAR_NORMAL_THRESHOLD = 0.28

# MediaPipe Setup
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, refine_landmarks=True)

def calculate_ear(landmarks, eye_indices):
    v1 = dist.euclidean(landmarks[eye_indices[1]], landmarks[eye_indices[5]])
    v2 = dist.euclidean(landmarks[eye_indices[2]], landmarks[eye_indices[4]])
    h = dist.euclidean(landmarks[eye_indices[0]], landmarks[eye_indices[3]])
    return (v1 + v2) / (2.0 * h)

def calculate_mar(landmarks):
    v = dist.euclidean(landmarks[13], landmarks[14])
    h = dist.euclidean(landmarks[78], landmarks[308])
    return v / h

def process_video(video_path, label):
    # Ensure the output folder is strictly the action name
    final_output_dir = os.path.join(output_base_path, label)
    os.makedirs(final_output_dir, exist_ok=True)
    
    cap = cv2.VideoCapture(video_path)
    video_name = os.path.basename(video_path).split('.')[0]
    
    saved_count = 0
    frame_idx = 0

    while cap.isOpened():
        success, frame = cap.read()
        if not success: break

        # 1. SKIP FRAMES to reduce volume
        if frame_idx % FRAME_SKIP != 0:
            frame_idx += 1
            continue
            
        # 2. OPTIONAL CAP
        if saved_count >= MAX_FRAMES_PER_VIDEO:
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(rgb_frame)

        if results.multi_face_landmarks:
            mesh_coords = [(lm.x, lm.y) for lm in results.multi_face_landmarks[0].landmark]
            
            ear_left = calculate_ear(mesh_coords, [33, 160, 158, 133, 153, 144])
            ear_right = calculate_ear(mesh_coords, [362, 385, 387, 263, 373, 380])
            avg_ear = (ear_left + ear_right) / 2.0
            mar = calculate_mar(mesh_coords)

            save_this_frame = False
            
            # Logic for filtering
            if label == 'Yawning' and mar > MAR_YAWN_THRESHOLD:
                save_this_frame = True
            elif label == 'Microsleep' and avg_ear < EAR_SLEEP_THRESHOLD:
                save_this_frame = True
            elif label == 'Talking' and MAR_TALK_THRESHOLD < mar < MAR_YAWN_THRESHOLD:
                save_this_frame = True
            elif label == 'Normal' and avg_ear > EAR_NORMAL_THRESHOLD and mar < 0.2:
                save_this_frame = True

            if save_this_frame:
                frame_resized = cv2.resize(frame, IMG_SIZE)
                # Use video name + frame index to ensure unique filenames
                filename = f"{video_name}_f{frame_idx}.jpg"
                cv2.imwrite(os.path.join(final_output_dir, filename), frame_resized)
                saved_count += 1
        
        frame_idx += 1
    cap.release()
    return saved_count

# --- Main Loop ---
# Define your target labels explicitly to ensure mapping is correct
valid_labels = ['Yawning', 'Microsleep', 'Talking', 'Normal']

for root, dirs, files in os.walk(input_base_path):
    for file in files:
        if file.lower().endswith(('.mp4', '.avi')):
            # Logic to extract label: adjust parts index if your folder structure is deeper
            parts = os.path.normpath(root).split(os.sep)
            
            # Find which of our target labels is in the path
            current_label = None
            for label in valid_labels:
                if label.lower() in [p.lower() for p in parts]:
                    current_label = label
                    break
            
            if current_label:
                print(f"Processing: {file} | Target Category: {current_label}")
                n = process_video(os.path.join(root, file), current_label)
                print(f"--> Saved {n} images to {current_label}/")
            else:
                print(f"Skipping {file}: No valid label found in path.")

print("Extraction Complete!")

Processing: P1042751_720.mp4 | Target Category: Microsleep
--> Saved 32 images to Microsleep/
Processing: P1042756_720.mp4 | Target Category: Microsleep
--> Saved 83 images to Microsleep/
Processing: P1042757_720.mp4 | Target Category: Microsleep
--> Saved 14 images to Microsleep/
Processing: P1042762_720.mp4 | Target Category: Microsleep
--> Saved 100 images to Microsleep/
Processing: P1042767_720.mp4 | Target Category: Microsleep
--> Saved 74 images to Microsleep/
Processing: P1042772_720.mp4 | Target Category: Microsleep
--> Saved 19 images to Microsleep/
Processing: P1042786_720.mp4 | Target Category: Microsleep
--> Saved 57 images to Microsleep/
Processing: P1042787_720.mp4 | Target Category: Microsleep
--> Saved 12 images to Microsleep/
Processing: P1042793_720.mp4 | Target Category: Microsleep
--> Saved 87 images to Microsleep/
Processing: P1042797_720.mp4 | Target Category: Microsleep
--> Saved 100 images to Microsleep/
Processing: P1043067_720.mp4 | Target Category: Microsleep